In [ ]:
import fitz
import camelot
import pandas as pd
from tabula.io import read_pdf
import os

In [ ]:
# --- Configuration ---
# PDF_PATH = "docs/summary_pages.pdf"
PDF_PATH = "docs/B & E FOODS PTY LTD.pdf"

CAMELOT_OUTPUT_EXCEL = f"extracted_tables_camelot_{os.path.splitext(os.path.basename(PDF_PATH))[0]}.xlsx"
TABULA_OUTPUT_EXCEL = f"extracted_tables_tabula_{os.path.splitext(os.path.basename(PDF_PATH))[0]}.xlsx"

In [41]:
# --- 1. PyMuPDF (fitz) Extraction ---
def run_pymupdf_extraction(pdf_path):
    """
    Extracts text, blocks, words, and attempts to find tables using PyMuPDF.
    """
    try:
        doc = fitz.open(pdf_path)
        print(f"Document: '{os.path.basename(pdf_path)}', Pages: {doc.page_count}")

        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            print(f"--- Page {page_num + 1} ---")

            # 1.1 Extract plain text
            # text_simple = page.get_text("text")

            # 1.2 Basic Table Detection (PyMuPDF's built-in capability)
            tables = page.find_tables()
            if tables.tables:  # Check if any tables were found
                print(f"Found {len(tables.tables)} table(s) on page {page_num + 1} using PyMuPDF finder.")
                for i, tab_finder_result in enumerate(tables):
                    print(f" Table {i+1} BBox: {tab_finder_result.bbox}")
                    # To extract data from a found table:
                    table_data = tab_finder_result.extract()
                    print(f"Table {i+1} Data (first few rows): {table_data}") # Example
            else:
                print(f"No tables found on page {page_num + 1} using PyMuPDF's basic finder.")

        doc.close()

    except FileNotFoundError:
        print(f"Error: The file '{pdf_path}' was not found.")
    except Exception as e:
        print(f"An error occurred during PyMuPDF extraction: {e}")
    finally:
        print("--- PyMuPDF (fitz) extraction finished ---")

In [42]:
# --- 2. Camelot Table Extraction ---
def run_camelot_extraction(pdf_path, output_excel_path):
    """
    Extracts tables using Camelot (both lattice and stream methods)
    and saves them to an Excel file.
    """
    if not os.path.exists(pdf_path):
        print(f"Error: The file '{pdf_path}' was not found for Camelot.")
        return

    try:
        print("Attempting Camelot 'lattice' extraction...")
        # suppress_stdout=False can be noisy, True to make it quieter
        lattice_tables = camelot.read_pdf(pdf_path, pages='all', flavor='lattice', suppress_stdout=True)
        print(f"Camelot 'lattice' found {len(lattice_tables)} tables.")

        print("Attempting Camelot 'stream' extraction...")
        stream_tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream', suppress_stdout=True)
        print(f"Camelot 'stream' found {len(stream_tables)} tables.")

        if not lattice_tables and not stream_tables:
            print("No tables found by Camelot.")
        else:
            with pd.ExcelWriter(output_excel_path) as writer:
                # Save lattice tables
                for i, table in enumerate(lattice_tables):
                    sheet_name = f'Lattice_{i+1}'
                    table.df.to_excel(writer, sheet_name=sheet_name, index=False)
                    print(f"  Saved Camelot 'lattice' table {i+1} to sheet '{sheet_name}'")

                # Save stream tables
                for i, table in enumerate(stream_tables):
                    sheet_name = f'Stream_{i+1}'
                    table.df.to_excel(writer, sheet_name=sheet_name, index=False)
                    print(f"  Saved Camelot 'stream' table {i+1} to sheet '{sheet_name}'")
            print(f"All Camelot tables have been saved to '{output_excel_path}'")

    except Exception as e:
        print(f"An error occurred during Camelot extraction: {e}")

In [ ]:
# --- 3. Tabula-py Table Extraction ---
def run_tabula_extraction(pdf_path, output_excel_path):
    """
    Extracts tables using Tabula-py (trying both lattice and stream)
    and saves them to an Excel file.
    """
    print(f"--- Starting Tabula-py table extraction for: {pdf_path} ---")
    if not os.path.exists(pdf_path):
        print(f"Error: The file '{pdf_path}' was not found for Tabula.")
        print("--- Tabula-py extraction skipped ---")
        return

    try:
        # Read tables from PDF using Tabula
        # guess=False forces use of specified lattice/stream
        # pandas_options={'header': None} to avoid auto-detecting header
        print("Attempting Tabula extraction (lattice=True, stream=True)...")
        tables = read_pdf(
            pdf_path,
            pages='all',
            multiple_tables=True,
            lattice=True,
            stream=True,
            guess=False,
            pandas_options={'header': None}
        )

        if not tables:
            print("No tables found by Tabula-py.")
        else:
            print(f"Tabula-py found {len(tables)} tables.")
            with pd.ExcelWriter(output_excel_path) as writer:
                for i, table_df in enumerate(tables):
                    if isinstance(table_df, pd.DataFrame):
                        sheet_name = f"Table_{i+1}"
                        table_df.to_excel(writer, sheet_name=sheet_name, index=False)
                        print(f"  Saved Tabula table {i+1} to sheet '{sheet_name}'")
                    else:
                        print(f"  Skipping item {i+1} from Tabula output as it's not a DataFrame.")

            print(f"All Tabula tables have been saved to '{output_excel_path}'")

    except Exception as e:
        print(f"An error occurred during Tabula-py extraction: {e}")
        print("Ensure Java is installed and in your system's PATH for Tabula-py.")
    finally:
        print("--- Tabula-py extraction finished ---")


In [44]:
# --- Main Execution ---
if __name__ == "__main__":
    print("=============================================")
    print("   PDF Extraction Testing Script   ")
    print("=============================================")
    print(f"Processing PDF: {PDF_PATH}")

    # Check if PDF exists before starting
    if not os.path.exists(PDF_PATH):
        print(f"FATAL ERROR: PDF file '{PDF_PATH}' not found. Please check the path.")
        print("Script will now exit.")
    else:
        # Test PyMuPDF
        run_pymupdf_extraction(PDF_PATH)

        # Test Camelot
        run_camelot_extraction(PDF_PATH, CAMELOT_OUTPUT_EXCEL)

        # Test Tabula-py
        run_tabula_extraction(PDF_PATH, TABULA_OUTPUT_EXCEL)

        print("=============================================")
        print("All extraction tests finished.")
        print(f"Camelot output (if any): {CAMELOT_OUTPUT_EXCEL}")
        print(f"Tabula output (if any): {TABULA_OUTPUT_EXCEL}")
        print("=============================================")


   PDF Extraction Testing Script   
Processing PDF: C:/Users/PK/Desktop/projects/financial_statement_analysis/docs/B & E FOODS PTY LTD.pdf
Document: 'B & E FOODS PTY LTD.pdf', Pages: 30
--- Page 1 ---
No tables found on page 1 using PyMuPDF's basic finder.
--- Page 2 ---
No tables found on page 2 using PyMuPDF's basic finder.
--- Page 3 ---
No tables found on page 3 using PyMuPDF's basic finder.
--- Page 4 ---
No tables found on page 4 using PyMuPDF's basic finder.
--- Page 5 ---
No tables found on page 5 using PyMuPDF's basic finder.
--- Page 6 ---
No tables found on page 6 using PyMuPDF's basic finder.
--- Page 7 ---
No tables found on page 7 using PyMuPDF's basic finder.
--- Page 8 ---
No tables found on page 8 using PyMuPDF's basic finder.
--- Page 9 ---
No tables found on page 9 using PyMuPDF's basic finder.
--- Page 10 ---
No tables found on page 10 using PyMuPDF's basic finder.
--- Page 11 ---
No tables found on page 11 using PyMuPDF's basic finder.
--- Page 12 ---
No tables fou

CropBox missing from /Page, defaulting to MediaBox


No tables found on page 30 using PyMuPDF's basic finder.
--- PyMuPDF (fitz) extraction finished ---
Attempting Camelot 'lattice' extraction...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Camelot 'lattice' found 0 tables.
Attempting Camelot 'stream' extraction...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Camelot 'stream' found 32 tables.
  Saved Camelot 'stream' table 1 to sheet 'Stream_1'
  Saved Camelot 'stream' table 2 to sheet 'Stream_2'
  Saved Camelot 'stream' table 3 to sheet 'Stream_3'
  Saved Camelot 'stream' table 4 to sheet 'Stream_4'
  Saved Camelot 'stream' table 5 to sheet 'Stream_5'
  Saved Camelot 'stream' table 6 to sheet 'Stream_6'
  Saved Camelot 'stream' table 7 to sheet 'Stream_7'
  Saved Camelot 'stream' table 8 to sheet 'Stream_8'
  Saved Camelot 'stream' table 9 to sheet 'Stream_9'
  Saved Camelot 'stream' table 10 to sheet 'Stream_10'
  Saved Camelot 'stream' table 11 to sheet 'Stream_11'
  Saved Camelot 'stream' table 12 to sheet 'Stream_12'
  Saved Camelot 'stream' table 13 to sheet 'Stream_13'
  Saved Camelot 'stream' table 14 to sheet 'Stream_14'
  Saved Camelot 'stream' table 15 to sheet 'Stream_15'
  Saved Camelot 'stream' table 16 to sheet 'Stream_16'
  Saved Camelot 'stream' table 17 to sheet 'Stream_17'
  Saved Camelot 'stream' table 18 to sheet 'Strea